In [8]:
# test for gmm-ubm on whale calls

from importlib import reload
import audio_datastore.audio_datastore as myads
reload(myads)
import os.path
from collections import Counter
import random
from python_speech_features import mfcc
import scipy.io.wavfile as wav
from spafe.utils import vis
import matplotlib.pyplot as plt
import misc.vad as vad
from sklearn.mixture import GaussianMixture, _gaussian_mixture
reload(vad)
import numpy as np
import librosa.feature as fe
reload(fe)
import librosa.util.utils as librosa_utils
import whale.setup.constants as const
from scipy.special import logsumexp


In [32]:
# helper functions

# [2] feature extraction
# 	• Normalize the audio
# 	• Use detectSpeech to remove nonspeech regions from the audio
# 	• Extract features from the audio
# 	• Normalize the features
#   * Apply cepstral mean normalization

eps = np.finfo(np.float64).eps

def helper_feature_extraction(raw_audio_file, norm = None):
    # read in file
    (signal_rate, signal) = wav.read(raw_audio_file)

    # normalise
    signal = librosa_utils.normalize(signal)

    # detect / vad, not working currently

    # fe
    mfcc_feats = fe.mfcc(y=signal,sr=const.SAMPLING_RATE, n_mfcc=13, n_fft=1024).T

    # feature normalisation and Cepstral mean subtraction (for channel noise)
    if norm:
        mfcc_feats = (mfcc_feats - norm.means) / norm.std
        mfcc_feats = mfcc_feats - np.mean(mfcc_feats)
        return mfcc_feats
    else:
        return mfcc_feats

In [6]:
datasetFolder = r"/Users/david/Documents/mastersCode/ubm/whale/setup/data_spliced"
ads = myads.AudioDatastore()
ads.populate(datasetFolder,include_sub_folders=True, label_source=True)
Counter(ads.labels).values()
adsTest, adsTrain = myads.split(ads, 15)
adsTrain, _ = myads.split(adsTrain, 13)

In [14]:
# extract all features and use to get feature normalization
all_features = []
for file in ads.files:
    feature = helper_feature_extraction(file)
    all_features.append(feature.T)

In [22]:
def get_norm_factors(all_f):
    means = []
    std = []
    for f in all_f:
        means.append(np.mean(f, axis=0))
        std.append(np.std(f, axis=0))

    means = np.array(means)
    means = np.mean(means, axis=0)

    std = np.array(std)
    std = np.mean(std, axis=0)

    class NormFactor:
        def __init__(self, m, s):
            self.means = m
            self.std = s

    return NormFactor(means, std)

In [23]:
# collect normalization factors
import pickle
# import helper_functions

normFactors = get_norm_factors(all_features)

In [24]:
# normalize features
all_features_norm = []
for i in range(len(all_features)):
    normalised = (all_features[i] - normFactors.means) / normFactors.std
    normalised = normalised - np.mean(normalised)
    all_features_norm.append(normalised)

In [35]:
# COLLECT TRAIN FEATURES

train_features = []
for i in range(len(adsTrain.files)):
    train_feature = helper_feature_extraction(adsTrain.files[i], normFactors)
    train_features.append(train_feature)

In [36]:
ubm = GaussianMixture(n_components=32, covariance_type='diag')
train_features_flattened = np.array([item for sublist in train_features for item in sublist])
ubm.fit(train_features_flattened)
print('sklearn model' + str(ubm.score(train_features[0])))

sklearn model-23.525702636759235


In [ ]:
# ENROLL

